2021.11.26 ~ 2022.02.28 => 주어진 데이터,
2022.03.04 ~ 13 그리고 03.18 ~ 20  => 검증 데이터

*최종 제출 방식))))))예보 데이터 시간 데이터 이용 => sum으로 일자 제출
분별 대체로 볼 때 0.2에서 0.4라고 한다면, 0.2 * 1440에서 0.4 * 1440 =>대략 300~500?

우선! 바운더리에 들어오면, 에너지 절감 타당한 모델을 만드는 게 핵심. 노가다 ㄴㄴ

*외부 데이터 방식)))))))관련 예보 데이터에 나와 있는 관측소만 이용해서 평균으로 보내서 하면 근거가 충분할 듯함
(주어진 게 그 관측소뿐이므로)

*주어진 데이터 방식))))열 공급량은 분=>시간 단위로 바뀔 때 합산해야함 (논리적으로 봤을 때, 시간 단위라면 분 단위의 누적이 되어야 맞는 거임. 의미적으로나 수치적으로나 둘 다.)
나머지 변수 => 일단 평균으로 (의미상,수치상 합산은 아닌듯..기온같은 경우..)

*공통))))outlier채우기, 중간에 없는 일자 뽑아서 외부,본 데이터 둘 다 삭제, 검증 데이터 3/15 삭제

*본 데이터12월 빠진 데이터 있음. (22,23,24), 외부 데이터 3월 15일 제거해야함

위는 따라서 외부 데이터에서 제거할 목록임

In [1]:
import pandas as pd
import numpy as np
from tqdm import tqdm
import os
import matplotlib.pyplot as plt
import seaborn as sns

import sklearn
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import KFold

from tensorflow import keras
from keras.models import Sequential
from keras import Input
from keras.layers import Dense
from tensorflow.keras.callbacks import ModelCheckpoint
from keras.models import load_model
from keras.callbacks import EarlyStopping

from lightgbm import LGBMRegressor
import random
import xgboost as xgb

# 데이터 로드

In [380]:
test1

,smart_farm.yy,smart_farm.mm,smart_farm.dd,smart_farm.hh,smart_farm.nn,smart_farm.insolation,smart_farm.out_tmperature,smart_farm.out_humidity,smart_farm.wind_speed,smart_farm.wind_direction,...,smart_farm.shield_tmperature_h,smart_farm.shield_energy_h,smart_farm.shield_energy_v,smart_farm.exhaust_fan,smart_farm.ceiling,smart_farm.floating_fan,smart_farm.fan_coil_b_site,smart_farm.ventilation_temperature_control,smart_farm.heating_temperature_set_up,smart_farm.heat_supply
0,2022,3,4,0,0,-4.0,2.0,76,0.0,5,...,0.0,0.0,0.0,0.0,0.0,1.0,1.0,28.0,18.0,NaN
1,2022,3,4,0,1,-4.1,2.0,76,0.4,5,...,0.0,0.0,0.0,0.0,0.0,1.0,1.0,28.0,18.0,NaN
2,2022,3,4,0,2,-4.1,1.9,76,0.0,5,...,0.0,0.0,0.0,0.0,0.0,1.0,1.0,28.0,18.0,NaN
3,2022,3,4,0,3,-4.1,1.9,76,0.0,5,...,0.0,0.0,0.0,0.0,0.0,1.0,1.0,28.0,18.0,NaN
4,2022,3,4,0,4,-4.2,1.9,76,0.0,5,...,0.0,0.0,0.0,0.0,0.0,1.0,1.0,28.0,18.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18715,2022,3,20,23,55,-3.0,6.2,61,0.4,344,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,28.0,18.0,NaN
18716,2022,3,20,23,56,-3.2,6.2,61,0.0,344,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,28.0,18.0,NaN
18717,2022,3,20,23,57,-3.2,6.1,61,0.4,344,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,28.0,18.0,NaN
18718,2022,3,20,23,58,-3.3,6.1,61,0.4,346,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,28.0,18.0,NaN


In [2]:
curr_dir = os.getcwd()
curr_dir

forders_tr = os.listdir(curr_dir+"\\data")
forders_tr

df = pd.read_csv(os.path.join(curr_dir+"\\data\\")+forders_tr[0])
df = df.drop(df.columns[0], axis=1)
df

df2022 = df[df['smart_farm.yy']==2022]
df2022
df202203 = df2022[df2022['smart_farm.mm']==3]
df202203

train = df.iloc[:132480]
train = train.reset_index()
train = train.drop('index', axis=1)

test = df.iloc[132480:]
test = test[test['smart_farm.dd']!=15]        # 15일자 제외
test = test.reset_index()
test = test.drop('index', axis=1)
test

# train1 = train.dropna(axis=0)
train1 = train.interpolate(method ='linear')     
# train 결측치 채운 이유=> 시간 별 데이터 중간중간 결측돼서 이를 이용해서 일자 예측하려면 채워줘야하기 때문.
test1 = test.interpolate(method ='linear') 

submission = pd.read_csv('2-1_검증데이터셋.csv')
submission1 = submission.copy()
submission1             # "년도 월 일 시 분 => 년도 월 일"  예측

,yy,mm,dd,heat_supply_day1,heat_supply_day2
0,2022,3,4,NaN,NaN
1,2022,3,5,NaN,NaN
2,2022,3,6,NaN,NaN
3,2022,3,7,NaN,NaN
4,2022,3,8,NaN,NaN
5,2022,3,9,NaN,NaN
6,2022,3,10,NaN,NaN
7,2022,3,11,NaN,NaN
8,2022,3,12,NaN,NaN
9,2022,3,13,NaN,NaN


In [3]:
train1[train1.columns[-1]].value_counts()

0.500000    28429
0.000000    25599
0.100000    23461
0.400000    11202
0.600000     9269
            ...  
0.344715        1
0.339024        1
0.333333        1
0.327642        1
0.400847        1
Name: smart_farm.heat_supply, Length: 507, dtype: int64

In [4]:
train1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 132480 entries, 0 to 132479
Data columns (total 23 columns):
 #   Column                                      Non-Null Count   Dtype  
---  ------                                      --------------   -----  
 0   smart_farm.yy                               132480 non-null  int64  
 1   smart_farm.mm                               132480 non-null  int64  
 2   smart_farm.dd                               132480 non-null  int64  
 3   smart_farm.hh                               132480 non-null  int64  
 4   smart_farm.nn                               132480 non-null  int64  
 5   smart_farm.insolation                       132480 non-null  float64
 6   smart_farm.out_tmperature                   132480 non-null  float64
 7   smart_farm.out_humidity                     132480 non-null  int64  
 8   smart_farm.wind_speed                       132480 non-null  float64
 9   smart_farm.wind_direction                   132480 non-null  int64  
 

# 외부 데이터 넣기

In [342]:
soil = pd.read_csv('OBS_AAOS_TIM_20220802210224.csv', encoding='cp949')       # 해당 시간 당 지점별 평균 내기
soil

,지점,지점명,일시,10CM 정시 토양수분(%),20CM 정시 토양수분(%),30CM 정시 토양수분(%),50CM 정시 토양수분(%)
0,119,수원,2021-11-26 00:00,4.5,25.9,24.1,26.6
1,119,수원,2021-11-26 01:00,4.5,25.9,24.1,26.6
2,119,수원,2021-11-26 02:00,4.6,25.9,24.1,26.6
3,119,수원,2021-11-26 03:00,4.6,25.9,24.1,26.6
4,119,수원,2021-11-26 04:00,4.5,25.9,24.1,26.6
...,...,...,...,...,...,...,...
28822,980,강정,2022-03-20 19:00,23.5,34.3,38.0,40.8
28823,980,강정,2022-03-20 20:00,23.6,34.3,38.0,40.8
28824,980,강정,2022-03-20 21:00,23.6,34.3,38.0,40.8
28825,980,강정,2022-03-20 22:00,23.6,34.8,38.8,40.8


In [343]:
stn = list(set(soil['지점'].values))
stn

[129, 258, 970, 972, 973, 974, 977, 978, 980, 119, 702]

In [344]:
# 인덱스 리셋, 지점/지점명/일시 컬럼 삭제 ==> 추후 일시 컬럼 복구(df간의 계산 위해)
soil_df = []
for i in range(len(stn)):
    df = soil[soil['지점']==stn[i]]
    df = df.reset_index()
    df = df.drop('index', axis=1)
    df = df.drop(['지점', '지점명', '일시'], axis=1)
    soil_df.append(df)
soil_df[0]

,10CM 정시 토양수분(%),20CM 정시 토양수분(%),30CM 정시 토양수분(%),50CM 정시 토양수분(%)
0,15.6,16.8,28.3,35.0
1,15.5,16.6,28.1,35.0
2,15.5,16.6,28.1,35.0
3,15.6,16.6,28.1,35.0
4,15.6,16.5,28.1,35.0
...,...,...,...,...
2488,20.6,23.2,31.6,32.9
2489,20.6,23.3,31.5,32.9
2490,20.6,23.3,31.6,32.9
2491,20.7,23.2,31.6,32.9


In [345]:
a = []
for i in range(len(soil_)):
    a.append(len(soil_[i]['일시'].values))     # 중간에 없는 일자의 관측소도 있음.(2710가 최대=>나머지는 0으로 채움.)
a

[2493, 2710, 2710, 2352, 2710, 2331, 2710, 2709, 2710, 2710, 2682]

In [346]:
# stn     # 129, 972, 974, 978, 702 => 전처리할 파일 _1

In [347]:
ind_1 = [index for index,value in enumerate(a) if value != max(a)]
ind_1

[0, 3, 5, 7, 10]

In [348]:
stn_1 = []
for i in range(len(ind_1)):
    stn_1.append(stn[ind_1[i]])
stn_1

[129, 972, 974, 978, 702]

In [349]:
for i in range(len(stn)):
    df = soil[soil['지점']==stn[i]]
    df.to_csv(os.path.join('stn\\'+str(stn[i])+'.csv'), index=False)
    
## 임의로 비어 있는 부분 nan 표기 엑셀 vlookup이용해서
## 그 후 0으로 채우고 => 평균화

In [350]:
stn[ind_1[1]]

972

In [351]:
# 참고로, 앞부분 NAN은 안 중요 => 모두 0으로 채워도 됨. 뒷부분 열만 쓰므로
for i in range(len(stn_1)):
    df = pd.read_csv(os.path.join('stn\\'+str(stn_1[i])+'_1.csv'), encoding='cp949')
    df = df.drop(list(df.columns[2:8]), axis=1) 
    df = df.fillna(0)
    df = df.drop(['지점', '지점명', '일시.1'], axis=1)
    df.columns = soil_df[1].columns
    soil_df[ind_1[i]] = df

In [352]:
# 일자 저장
df = pd.read_csv(os.path.join('stn\\'+str(stn_1[0])+'_1.csv'), encoding='cp949')
day = df['일시.1'].values
day

array(['2021-11-26 0:00', '2021-11-26 1:00', '2021-11-26 2:00', ...,
       '2022-03-20 21:00', '2022-03-20 22:00', '2022-03-20 23:00'],
      dtype=object)

In [353]:
# 하나의 데이터 프레임으로(빈 값 하나 만들어서)
soil_df1 = soil_df[0].copy()
for i in range(len(soil_df[0].columns)):
    soil_df1[soil_df[0].columns[i]] = 0
soil_df1

for i in range(len(soil_df)):
    soil_df1 = soil_df1 + soil_df[i]
soil_df1 = soil_df1/len(soil_df)
soil_df1  

,10CM 정시 토양수분(%),20CM 정시 토양수분(%),30CM 정시 토양수분(%),50CM 정시 토양수분(%)
0,21.400000,22.190909,21.800000,24.372727
1,21.327273,22.172727,21.900000,24.381818
2,21.281818,22.127273,21.745455,24.372727
3,21.227273,22.109091,21.718182,24.372727
4,21.163636,22.072727,21.700000,24.363636
...,...,...,...,...
2705,20.218182,21.172727,19.836364,22.627273
2706,20.172727,21.109091,19.800000,22.627273
2707,20.190909,21.072727,19.809091,22.609091
2708,20.136364,21.045455,19.854545,22.618182


In [354]:
soil_df1['일자'] = day
soil_df1

,10CM 정시 토양수분(%),20CM 정시 토양수분(%),30CM 정시 토양수분(%),50CM 정시 토양수분(%),일자
0,21.400000,22.190909,21.800000,24.372727,2021-11-26 0:00
1,21.327273,22.172727,21.900000,24.381818,2021-11-26 1:00
2,21.281818,22.127273,21.745455,24.372727,2021-11-26 2:00
3,21.227273,22.109091,21.718182,24.372727,2021-11-26 3:00
4,21.163636,22.072727,21.700000,24.363636,2021-11-26 4:00
...,...,...,...,...,...
2705,20.218182,21.172727,19.836364,22.627273,2022-03-20 19:00
2706,20.172727,21.109091,19.800000,22.627273,2022-03-20 20:00
2707,20.190909,21.072727,19.809091,22.609091,2022-03-20 21:00
2708,20.136364,21.045455,19.854545,22.618182,2022-03-20 22:00


In [355]:
# 본 데이터12월 빠진 데이터 있음. (22,23,24)
# 외부 데이터 3월 15일 제거해야함

In [356]:
d = soil_df1[soil_df1['일자'] == '2021-12-22 0:00'].index[0]
d1 = soil_df1[soil_df1['일자'] == '2022-03-15 0:00'].index[0]
# print(d1)

for i in range(d, d+24*3):
    soil_df1 = soil_df1.drop(i,axis=0)
    
for i in range(d1, d1+24):
    soil_df1 = soil_df1.drop(i,axis=0)
    
# soil_df1.iloc[d:d+24*3]
# soil_df1.iloc[d1:d1+24]
soil_df1 = soil_df1.reset_index()
soil_df1 = soil_df1.drop('index', axis=1)
soil_df1

,10CM 정시 토양수분(%),20CM 정시 토양수분(%),30CM 정시 토양수분(%),50CM 정시 토양수분(%),일자
0,21.400000,22.190909,21.800000,24.372727,2021-11-26 0:00
1,21.327273,22.172727,21.900000,24.381818,2021-11-26 1:00
2,21.281818,22.127273,21.745455,24.372727,2021-11-26 2:00
3,21.227273,22.109091,21.718182,24.372727,2021-11-26 3:00
4,21.163636,22.072727,21.700000,24.363636,2021-11-26 4:00
...,...,...,...,...,...
2609,20.218182,21.172727,19.836364,22.627273,2022-03-20 19:00
2610,20.172727,21.109091,19.800000,22.627273,2022-03-20 20:00
2611,20.190909,21.072727,19.809091,22.609091,2022-03-20 21:00
2612,20.136364,21.045455,19.854545,22.618182,2022-03-20 22:00


In [394]:
d = soil_df1[soil_df1['일자'] == '2022-03-01 0:00'].index[0]
d1 = soil_df1[soil_df1['일자'] == '2022-03-04 0:00'].index[0]
soil_train = soil_df1.iloc[:d]
soil_test = soil_df1.iloc[d1:]

soil_test = soil_test.reset_index()
soil_test = soil_test.drop('index', axis=1)
soil_test

,10CM 정시 토양수분(%),20CM 정시 토양수분(%),30CM 정시 토양수분(%),50CM 정시 토양수분(%),일자
0,12.981818,16.572727,14.881818,18.472727,2022-03-04 0:00
1,12.890909,16.536364,14.881818,18.481818,2022-03-04 1:00
2,12.854545,16.509091,14.872727,18.481818,2022-03-04 2:00
3,12.745455,16.490909,14.890909,18.490909,2022-03-04 3:00
4,12.645455,16.472727,14.890909,18.490909,2022-03-04 4:00
...,...,...,...,...,...
374,20.218182,21.172727,19.836364,22.627273,2022-03-20 19:00
375,20.172727,21.109091,19.800000,22.627273,2022-03-20 20:00
376,20.190909,21.072727,19.809091,22.609091,2022-03-20 21:00
377,20.136364,21.045455,19.854545,22.618182,2022-03-20 22:00


In [392]:
soil_train.to_csv('train_prep.csv', index=False)
soil_test.to_csv('test_prep.csv', index=False)

In [359]:
# soil_train.to_csv('train_prep.csv', index=False)
i = pd.date_range('2021-11-26', periods=2352, freq='1H')
i

DatetimeIndex(['2021-11-26 00:00:00', '2021-11-26 01:00:00',
               '2021-11-26 02:00:00', '2021-11-26 03:00:00',
               '2021-11-26 04:00:00', '2021-11-26 05:00:00',
               '2021-11-26 06:00:00', '2021-11-26 07:00:00',
               '2021-11-26 08:00:00', '2021-11-26 09:00:00',
               ...
               '2022-03-03 14:00:00', '2022-03-03 15:00:00',
               '2022-03-03 16:00:00', '2022-03-03 17:00:00',
               '2022-03-03 18:00:00', '2022-03-03 19:00:00',
               '2022-03-03 20:00:00', '2022-03-03 21:00:00',
               '2022-03-03 22:00:00', '2022-03-03 23:00:00'],
              dtype='datetime64[ns]', length=2352, freq='H')

# 외부 데이터에도 없는 날짜 채워줘야함_train

In [360]:
i = pd.date_range('2021-11-26', periods=2280, freq='1H')         #2352
i
day1 = pd.DataFrame(i)
day1.columns = ['일자']
day1
day1.to_csv('day.csv', index=False)

In [366]:
soil_train1 = pd.read_csv('train_prep_1.csv', encoding='cp949')
soil_train1
soil_train1 = soil_train1.drop(list(soil_train1.columns[:6]), axis=1) 
# soil_train1 = soil_train1.fillna(0)
soil_train1 = soil_train1.drop(['일자.1'], axis=1)
soil_train1.columns = soil_df[1].columns
soil_train1.drop(2280, axis=0, inplace=True)

In [367]:
soil_train1 = soil_train1.interpolate(method ='linear')     # interpolation 적용하고 특정 날짜를 빼야 의미가 상통(전처리=>다시 날짜빼줘야함)
soil_train1

soil_train1['일자'] = i
soil_train1

,10CM 정시 토양수분(%),20CM 정시 토양수분(%),30CM 정시 토양수분(%),50CM 정시 토양수분(%),일자
0,21.400000,22.190909,21.800000,24.372727,2021-11-26 00:00:00
1,21.327273,22.172727,21.900000,24.381818,2021-11-26 01:00:00
2,21.281818,22.127273,21.745455,24.372727,2021-11-26 02:00:00
3,21.227273,22.109091,21.718182,24.372727,2021-11-26 03:00:00
4,21.163636,22.072727,21.700000,24.363636,2021-11-26 04:00:00
...,...,...,...,...,...
2275,12.381818,14.727273,13.590909,17.200000,2022-02-28 19:00:00
2276,12.318182,14.772727,13.609091,17.209091,2022-02-28 20:00:00
2277,12.290909,14.790909,13.618182,17.209091,2022-02-28 21:00:00
2278,12.227273,14.727273,13.627273,17.209091,2022-02-28 22:00:00


In [368]:
d = soil_train1[soil_train1['일자'] == '2021-12-22 0:00'].index[0]
# print(d1)

for i in range(d, d+24*3):
    soil_train1 = soil_train1.drop(i,axis=0)
    
soil_train1 = soil_train1.reset_index()
soil_train1 = soil_train1.drop('index', axis=1)
soil_train1

,10CM 정시 토양수분(%),20CM 정시 토양수분(%),30CM 정시 토양수분(%),50CM 정시 토양수분(%),일자
0,21.400000,22.190909,21.800000,24.372727,2021-11-26 00:00:00
1,21.327273,22.172727,21.900000,24.381818,2021-11-26 01:00:00
2,21.281818,22.127273,21.745455,24.372727,2021-11-26 02:00:00
3,21.227273,22.109091,21.718182,24.372727,2021-11-26 03:00:00
4,21.163636,22.072727,21.700000,24.363636,2021-11-26 04:00:00
...,...,...,...,...,...
2203,12.381818,14.727273,13.590909,17.200000,2022-02-28 19:00:00
2204,12.318182,14.772727,13.609091,17.209091,2022-02-28 20:00:00
2205,12.290909,14.790909,13.618182,17.209091,2022-02-28 21:00:00
2206,12.227273,14.727273,13.627273,17.209091,2022-02-28 22:00:00


In [369]:
## 외부데이터 날짜에 조차 아예 없는 값이 있다
# 11/26~30, 12/31,1/31,2/28,3/1~3/3 => 95(5+31+31+28) => 총 95번 동안 0~23 채워서 full date하면 됨 => interpolation 결측치 채움(엑셀)
95*24 - 3*24

2208

# 외부 데이터에도 없는 날짜 채워줘야함_test

In [397]:
ii = pd.date_range('2022-03-04', periods=17*24, freq='1H')         #2352
ii
day1 = pd.DataFrame(ii)
day1.columns = ['일자']
day1
# day1.to_csv('day_test.csv', index=False)
day1

,일자
0,2022-03-04 00:00:00
1,2022-03-04 01:00:00
2,2022-03-04 02:00:00
3,2022-03-04 03:00:00
4,2022-03-04 04:00:00
...,...
403,2022-03-20 19:00:00
404,2022-03-20 20:00:00
405,2022-03-20 21:00:00
406,2022-03-20 22:00:00


In [398]:
soil_test1 = pd.read_csv('test_prep_1.csv', encoding='cp949')
soil_test1
soil_test1 = soil_test1.drop(list(soil_test1.columns[:6]), axis=1) 
# soil_train1 = soil_train1.fillna(0)
soil_test1 = soil_test1.drop(['일자.1'], axis=1)
soil_test1.columns = soil_df[1].columns
# soil_test1.drop(2280, axis=0, inplace=True)
soil_test1

,10CM 정시 토양수분(%),20CM 정시 토양수분(%),30CM 정시 토양수분(%),50CM 정시 토양수분(%)
0,12.981818,16.572727,14.881818,18.472727
1,12.890909,16.536364,14.881818,18.481818
2,12.854545,16.509091,14.872727,18.481818
3,12.745455,16.490909,14.890909,18.490909
4,12.645455,16.472727,14.890909,18.490909
...,...,...,...,...
403,20.218182,21.172727,19.836364,22.627273
404,20.172727,21.109091,19.800000,22.627273
405,20.190909,21.072727,19.809091,22.609091
406,20.136364,21.045455,19.854545,22.618182


In [399]:
soil_test1 = soil_test1.interpolate(method ='linear')     # interpolation 적용하고 특정 날짜를 빼야 의미가 상통(전처리=>다시 날짜빼줘야함)
soil_test1

soil_test1['일자'] = ii
soil_test1

,10CM 정시 토양수분(%),20CM 정시 토양수분(%),30CM 정시 토양수분(%),50CM 정시 토양수분(%),일자
0,12.981818,16.572727,14.881818,18.472727,2022-03-04 00:00:00
1,12.890909,16.536364,14.881818,18.481818,2022-03-04 01:00:00
2,12.854545,16.509091,14.872727,18.481818,2022-03-04 02:00:00
3,12.745455,16.490909,14.890909,18.490909,2022-03-04 03:00:00
4,12.645455,16.472727,14.890909,18.490909,2022-03-04 04:00:00
...,...,...,...,...,...
403,20.218182,21.172727,19.836364,22.627273,2022-03-20 19:00:00
404,20.172727,21.109091,19.800000,22.627273,2022-03-20 20:00:00
405,20.190909,21.072727,19.809091,22.609091,2022-03-20 21:00:00
406,20.136364,21.045455,19.854545,22.618182,2022-03-20 22:00:00


In [403]:
d = soil_test1[soil_test1['일자'] == '2022-03-14 0:00'].index[0]
soil_test1.iloc[d:d+24*4]

for i in range(d, d+24*4):
    soil_test1 = soil_test1.drop(i,axis=0)
    
soil_test1 = soil_test1.reset_index()
soil_test1 = soil_test1.drop('index', axis=1)
soil_test1

,10CM 정시 토양수분(%),20CM 정시 토양수분(%),30CM 정시 토양수분(%),50CM 정시 토양수분(%),일자
0,12.981818,16.572727,14.881818,18.472727,2022-03-04 00:00:00
1,12.890909,16.536364,14.881818,18.481818,2022-03-04 01:00:00
2,12.854545,16.509091,14.872727,18.481818,2022-03-04 02:00:00
3,12.745455,16.490909,14.890909,18.490909,2022-03-04 03:00:00
4,12.645455,16.472727,14.890909,18.490909,2022-03-04 04:00:00
...,...,...,...,...,...
307,20.218182,21.172727,19.836364,22.627273,2022-03-20 19:00:00
308,20.172727,21.109091,19.800000,22.627273,2022-03-20 20:00:00
309,20.190909,21.072727,19.809091,22.609091,2022-03-20 21:00:00
310,20.136364,21.045455,19.854545,22.618182,2022-03-20 22:00:00


In [ ]:
#### 외부데이터로 뭉친 train,test 따로 저장 => 드라이브

# 본 데이터 시간별로 만들기

In [327]:
col = ['smart_farm.shield_light_h',\
       'smart_farm.shield_tmperature_h', 'smart_farm.shield_energy_h',\
       'smart_farm.shield_energy_v', 'smart_farm.exhaust_fan',\
       'smart_farm.ceiling', 'smart_farm.floating_fan',\
       'smart_farm.fan_coil_b_site',\
       'smart_farm.ventilation_temperature_control',\
       'smart_farm.heating_temperature_set_up']

col

['smart_farm.shield_light_h',
 'smart_farm.shield_tmperature_h',
 'smart_farm.shield_energy_h',
 'smart_farm.shield_energy_v',
 'smart_farm.exhaust_fan',
 'smart_farm.ceiling',
 'smart_farm.floating_fan',
 'smart_farm.fan_coil_b_site',
 'smart_farm.ventilation_temperature_control',
 'smart_farm.heating_temperature_set_up']

In [328]:
train1

,smart_farm.yy,smart_farm.mm,smart_farm.dd,smart_farm.hh,smart_farm.nn,smart_farm.insolation,smart_farm.out_tmperature,smart_farm.out_humidity,smart_farm.wind_speed,smart_farm.wind_direction,...,smart_farm.shield_tmperature_h,smart_farm.shield_energy_h,smart_farm.shield_energy_v,smart_farm.exhaust_fan,smart_farm.ceiling,smart_farm.floating_fan,smart_farm.fan_coil_b_site,smart_farm.ventilation_temperature_control,smart_farm.heating_temperature_set_up,smart_farm.heat_supply
0,2021,11,26,0,0,-2.7,5.8,67,0.0,325,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,28.0,18.0,0.9
1,2021,11,26,0,1,-2.9,5.8,67,0.4,344,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,28.0,18.0,0.9
2,2021,11,26,0,2,-3.0,5.7,67,0.9,105,...,0.0,0.0,0.0,0.0,0.0,1.0,1.0,28.0,18.0,0.9
3,2021,11,26,0,3,-3.0,5.7,67,1.3,123,...,0.0,0.0,0.0,0.0,0.0,1.0,1.0,28.0,18.0,0.8
4,2021,11,26,0,4,-2.8,5.7,67,0.9,123,...,0.0,0.0,0.0,0.0,0.0,1.0,1.0,28.0,18.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
132475,2022,2,28,23,55,-2.7,4.1,51,0.0,57,...,0.0,0.0,0.0,0.0,0.0,1.0,1.0,28.0,18.0,0.9
132476,2022,2,28,23,56,-2.7,4.1,52,0.4,57,...,0.0,0.0,0.0,0.0,0.0,1.0,1.0,28.0,18.0,0.8
132477,2022,2,28,23,57,-2.7,4.1,52,0.4,40,...,0.0,0.0,0.0,0.0,0.0,1.0,1.0,28.0,18.0,0.8
132478,2022,2,28,23,58,-2.6,4.1,52,0.4,36,...,0.0,0.0,0.0,0.0,0.0,1.0,1.0,28.0,18.0,0.8


In [255]:
132480/60

2208.0

In [256]:
train_ = pd.DataFrame()
for k in tqdm(range(len(col))):
    h = []
    for i in range(int(len(train1)/60)):      # 분 -> 시간 단위 바꾸기 위해
        h.append(np.mean(train1[col[k]][60*i:60*(i+1)]))
    train_[col[k]] = h

    
h = []
for i in range(int(len(train1)/60)):
    h.append(np.mean(train1['smart_farm.heat_supply'][60*i:60*(i+1)]))
train_['smart_farm.heat_supply'] = h
train_        

100%|██████████████████████████████████████████████████████████████████████████████████| 10/10 [00:05<00:00,  1.86it/s]


,smart_farm.shield_light_h,smart_farm.shield_tmperature_h,smart_farm.shield_energy_h,smart_farm.shield_energy_v,smart_farm.exhaust_fan,smart_farm.ceiling,smart_farm.floating_fan,smart_farm.fan_coil_b_site,smart_farm.ventilation_temperature_control,smart_farm.heating_temperature_set_up,smart_farm.heat_supply
0,0.0,0.00,0.0,0.0,0.0,0.0,1.000000,0.600000,28.0,18.000,0.780000
1,0.0,0.00,0.0,0.0,0.0,0.0,1.000000,0.650000,28.0,18.000,0.811667
2,0.0,0.00,0.0,0.0,0.0,0.0,1.000000,0.683333,28.0,18.000,0.750000
3,0.0,0.00,0.0,0.0,0.0,0.0,1.000000,0.750000,28.0,18.000,0.766667
4,0.0,0.00,0.0,0.0,0.0,0.0,1.000000,0.783333,28.0,18.000,0.516667
...,...,...,...,...,...,...,...,...,...,...,...
2203,0.0,61.75,0.0,0.0,0.0,0.0,1.000000,0.000000,28.0,16.750,0.100000
2204,0.0,0.00,0.0,0.0,0.0,0.0,0.983333,0.000000,28.0,15.140,0.100000
2205,0.0,0.00,0.0,0.0,0.0,0.0,0.983333,0.150000,28.0,15.390,0.246667
2206,0.0,0.00,0.0,0.0,0.0,0.0,1.000000,0.833333,28.0,17.235,0.608333


In [419]:
train_real = pd.concat([soil_train1, train_], axis=1)
train_real

,10CM 정시 토양수분(%),20CM 정시 토양수분(%),30CM 정시 토양수분(%),50CM 정시 토양수분(%),일자,smart_farm.shield_light_h,smart_farm.shield_tmperature_h,smart_farm.shield_energy_h,smart_farm.shield_energy_v,smart_farm.exhaust_fan,smart_farm.ceiling,smart_farm.floating_fan,smart_farm.fan_coil_b_site,smart_farm.ventilation_temperature_control,smart_farm.heating_temperature_set_up,smart_farm.heat_supply
0,21.400000,22.190909,21.800000,24.372727,2021-11-26 00:00:00,0.0,0.00,0.0,0.0,0.0,0.0,1.000000,0.600000,28.0,18.000,0.780000
1,21.327273,22.172727,21.900000,24.381818,2021-11-26 01:00:00,0.0,0.00,0.0,0.0,0.0,0.0,1.000000,0.650000,28.0,18.000,0.811667
2,21.281818,22.127273,21.745455,24.372727,2021-11-26 02:00:00,0.0,0.00,0.0,0.0,0.0,0.0,1.000000,0.683333,28.0,18.000,0.750000
3,21.227273,22.109091,21.718182,24.372727,2021-11-26 03:00:00,0.0,0.00,0.0,0.0,0.0,0.0,1.000000,0.750000,28.0,18.000,0.766667
4,21.163636,22.072727,21.700000,24.363636,2021-11-26 04:00:00,0.0,0.00,0.0,0.0,0.0,0.0,1.000000,0.783333,28.0,18.000,0.516667
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2203,12.381818,14.727273,13.590909,17.200000,2022-02-28 19:00:00,0.0,61.75,0.0,0.0,0.0,0.0,1.000000,0.000000,28.0,16.750,0.100000
2204,12.318182,14.772727,13.609091,17.209091,2022-02-28 20:00:00,0.0,0.00,0.0,0.0,0.0,0.0,0.983333,0.000000,28.0,15.140,0.100000
2205,12.290909,14.790909,13.618182,17.209091,2022-02-28 21:00:00,0.0,0.00,0.0,0.0,0.0,0.0,0.983333,0.150000,28.0,15.390,0.246667
2206,12.227273,14.727273,13.627273,17.209091,2022-02-28 22:00:00,0.0,0.00,0.0,0.0,0.0,0.0,1.000000,0.833333,28.0,17.235,0.608333


In [420]:
# train_real.to_csv('train_real.csv', index=False)

# 검증 데이터 일자별 + 외부 셋 넣기

In [371]:
submission1

,yy,mm,dd,heat_supply_day1,heat_supply_day2
0,2022,3,4,NaN,NaN
1,2022,3,5,NaN,NaN
2,2022,3,6,NaN,NaN
3,2022,3,7,NaN,NaN
4,2022,3,8,NaN,NaN
5,2022,3,9,NaN,NaN
6,2022,3,10,NaN,NaN
7,2022,3,11,NaN,NaN
8,2022,3,12,NaN,NaN
9,2022,3,13,NaN,NaN


In [404]:
test_ = pd.DataFrame()
for k in tqdm(range(len(col))):
    h = []
    for i in range(int(len(test1)/60)):      # 분 -> 시간 단위 바꾸기 위해
        h.append(np.mean(test1[col[k]][60*i:60*(i+1)]))
    test_[col[k]] = h

    
h = []
for i in range(int(len(test1)/60)):
    h.append(np.mean(test1['smart_farm.heat_supply'][60*i:60*(i+1)]))
test_['smart_farm.heat_supply'] = h
test_        

100%|██████████████████████████████████████████████████████████████████████████████████| 10/10 [00:00<00:00, 31.24it/s]


,smart_farm.shield_light_h,smart_farm.shield_tmperature_h,smart_farm.shield_energy_h,smart_farm.shield_energy_v,smart_farm.exhaust_fan,smart_farm.ceiling,smart_farm.floating_fan,smart_farm.fan_coil_b_site,smart_farm.ventilation_temperature_control,smart_farm.heating_temperature_set_up,smart_farm.heat_supply
0,0.0,0.000000,0.0,0.0,0.0,0.0,1.000000,0.866667,28.000000,18.000000,NaN
1,0.0,0.000000,0.0,0.0,0.0,0.0,1.000000,0.733333,28.000000,18.000000,NaN
2,0.0,0.000000,0.0,0.0,0.0,0.0,1.000000,0.866667,28.000000,18.000000,NaN
3,0.0,0.000000,0.0,0.0,0.0,0.0,1.000000,0.816667,28.000000,18.000000,NaN
4,0.0,0.000000,0.0,0.0,0.0,0.0,1.000000,0.866667,28.000000,18.000000,NaN
...,...,...,...,...,...,...,...,...,...,...,...
307,0.0,91.316667,0.0,0.0,0.0,0.0,1.000000,0.000000,27.866667,17.383333,NaN
308,0.0,0.200000,0.0,0.0,0.0,0.0,0.983333,0.000000,28.000000,15.478333,NaN
309,0.0,0.000000,0.0,0.0,0.0,0.0,0.966667,0.150000,28.000000,15.095000,NaN
310,0.0,0.000000,0.0,0.0,0.0,0.0,1.000000,0.616667,28.000000,16.616667,NaN


In [421]:
test_real = pd.concat([soil_test1, test_], axis=1)
test_real

,10CM 정시 토양수분(%),20CM 정시 토양수분(%),30CM 정시 토양수분(%),50CM 정시 토양수분(%),일자,smart_farm.shield_light_h,smart_farm.shield_tmperature_h,smart_farm.shield_energy_h,smart_farm.shield_energy_v,smart_farm.exhaust_fan,smart_farm.ceiling,smart_farm.floating_fan,smart_farm.fan_coil_b_site,smart_farm.ventilation_temperature_control,smart_farm.heating_temperature_set_up,smart_farm.heat_supply
0,12.981818,16.572727,14.881818,18.472727,2022-03-04 00:00:00,0.0,0.000000,0.0,0.0,0.0,0.0,1.000000,0.866667,28.000000,18.000000,NaN
1,12.890909,16.536364,14.881818,18.481818,2022-03-04 01:00:00,0.0,0.000000,0.0,0.0,0.0,0.0,1.000000,0.733333,28.000000,18.000000,NaN
2,12.854545,16.509091,14.872727,18.481818,2022-03-04 02:00:00,0.0,0.000000,0.0,0.0,0.0,0.0,1.000000,0.866667,28.000000,18.000000,NaN
3,12.745455,16.490909,14.890909,18.490909,2022-03-04 03:00:00,0.0,0.000000,0.0,0.0,0.0,0.0,1.000000,0.816667,28.000000,18.000000,NaN
4,12.645455,16.472727,14.890909,18.490909,2022-03-04 04:00:00,0.0,0.000000,0.0,0.0,0.0,0.0,1.000000,0.866667,28.000000,18.000000,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
307,20.218182,21.172727,19.836364,22.627273,2022-03-20 19:00:00,0.0,91.316667,0.0,0.0,0.0,0.0,1.000000,0.000000,27.866667,17.383333,NaN
308,20.172727,21.109091,19.800000,22.627273,2022-03-20 20:00:00,0.0,0.200000,0.0,0.0,0.0,0.0,0.983333,0.000000,28.000000,15.478333,NaN
309,20.190909,21.072727,19.809091,22.609091,2022-03-20 21:00:00,0.0,0.000000,0.0,0.0,0.0,0.0,0.966667,0.150000,28.000000,15.095000,NaN
310,20.136364,21.045455,19.854545,22.618182,2022-03-20 22:00:00,0.0,0.000000,0.0,0.0,0.0,0.0,1.000000,0.616667,28.000000,16.616667,NaN


In [422]:
# test_real.to_csv('test_real.csv', index=False)

# Model1

In [409]:
train_real.columns

Index(['10CM 정시 토양수분(%)', '20CM 정시 토양수분(%)', '30CM 정시 토양수분(%)',
       '50CM 정시 토양수분(%)', '일자', 'smart_farm.shield_light_h',
       'smart_farm.shield_tmperature_h', 'smart_farm.shield_energy_h',
       'smart_farm.shield_energy_v', 'smart_farm.exhaust_fan',
       'smart_farm.ceiling', 'smart_farm.floating_fan',
       'smart_farm.fan_coil_b_site',
       'smart_farm.ventilation_temperature_control',
       'smart_farm.heating_temperature_set_up', 'smart_farm.heat_supply'],
      dtype='object')

In [435]:
max_depth = 5
n_estimators = 5000             # 400  #1000
lr = 0.01             # 0.017
num_leaves = 100
early_stopping_rounds = 300      # 100
verbose = True                  # True

In [436]:
X = train_real.drop(['10CM 정시 토양수분(%)', '20CM 정시 토양수분(%)', '30CM 정시 토양수분(%)',\
       '50CM 정시 토양수분(%)','일자','smart_farm.heat_supply'], axis=1)
y = train_real[['smart_farm.heat_supply']].values

X_train1, X_val1, y_train1, y_val1 = train_test_split(X, y, test_size=0.3)

X_test1 = test_real.drop(['10CM 정시 토양수분(%)', '20CM 정시 토양수분(%)', '30CM 정시 토양수분(%)',\
       '50CM 정시 토양수분(%)','일자','smart_farm.heat_supply'], axis=1)

In [437]:
lgbm_reg1 = LGBMRegressor(objective= "regression",
                     max_depth= max_depth,
                     n_estimators= n_estimators,
                     learning_rate= lr,
                     num_leaves = num_leaves)
    
lgbm_reg1.fit(X_train1, y_train1,
          eval_set=[(X_val1, y_val1)],
          eval_metric=["mse"],
          early_stopping_rounds=early_stopping_rounds,
          verbose=verbose) 

[1]	valid_0's l2: 0.0559578
[2]	valid_0's l2: 0.0555217
[3]	valid_0's l2: 0.0550911
[4]	valid_0's l2: 0.0546532
[5]	valid_0's l2: 0.0542261
[6]	valid_0's l2: 0.0538056
[7]	valid_0's l2: 0.0533955
[8]	valid_0's l2: 0.0529918
[9]	valid_0's l2: 0.0525974
[10]	valid_0's l2: 0.0522208
[11]	valid_0's l2: 0.0518551
[12]	valid_0's l2: 0.0514823
[13]	valid_0's l2: 0.051123
[14]	valid_0's l2: 0.0507642
[15]	valid_0's l2: 0.0504211
[16]	valid_0's l2: 0.0500924
[17]	valid_0's l2: 0.0497549
[18]	valid_0's l2: 0.0494289
[19]	valid_0's l2: 0.0491067
[20]	valid_0's l2: 0.0487967
[21]	valid_0's l2: 0.0484998
[22]	valid_0's l2: 0.0481952
[23]	valid_0's l2: 0.0479021
[24]	valid_0's l2: 0.047606
[25]	valid_0's l2: 0.0473251
[26]	valid_0's l2: 0.0470483
[27]	valid_0's l2: 0.0467805
[28]	valid_0's l2: 0.0465175
[29]	valid_0's l2: 0.0462631
[30]	valid_0's l2: 0.0460138
[31]	valid_0's l2: 0.0457591
[32]	valid_0's l2: 0.0455047
[33]	valid_0's l2: 0.0452712
[34]	valid_0's l2: 0.0450353
[35]	valid_0's l2: 0.0448

C:\Users\wjd72\anaconda3\envs\tensorflow_env\lib\site-packages\lightgbm\sklearn.py:726: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
C:\Users\wjd72\anaconda3\envs\tensorflow_env\lib\site-packages\lightgbm\sklearn.py:736: UserWarning: 'verbose' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose' argument is deprecated and will be removed in a future release of LightGBM. "
C:\Users\wjd72\anaconda3\envs\tensorflow_env\lib\site-packages\lightgbm\basic.py:179: UserWarning: Converting column-vector to 1d array
  _log_warning('Converting column-vector to 1d array')


LGBMRegressor(learning_rate=0.01, max_depth=5, n_estimators=5000,
              num_leaves=100, objective='regression')

In [438]:
pred_ = lgbm_reg1.predict(X_test1)
pred_

array([0.47098917, 0.45886182, 0.47098917, 0.46718905, 0.47098917,
       0.47979441, 0.4799896 , 0.43640677, 0.40516142, 0.17695617,
       0.42714721, 0.0810255 , 0.12594829, 0.1658194 , 0.13207141,
       0.11103438, 0.08237237, 0.17390152, 0.13339177, 0.23160996,
       0.18341005, 0.30847611, 0.35091014, 0.34489344, 0.34693258,
       0.34693258, 0.34693258, 0.40827027, 0.40827027, 0.47979441,
       0.4799896 , 0.43640677, 0.25953128, 0.0442472 , 0.0375833 ,
       0.11493298, 0.12687581, 0.15354274, 0.13069816, 0.07197344,
       0.07046981, 0.19614664, 0.32170719, 0.39466955, 0.30847611,
       0.38657649, 0.54455912, 0.48434712, 0.48434712, 0.45964392,
       0.48434712, 0.48434712, 0.48434712, 0.47979441, 0.4799896 ,
       0.43640677, 0.27849242, 0.07527633, 0.12474855, 0.12367149,
       0.18728626, 0.17344485, 0.12683155, 0.12683155, 0.1933445 ,
       0.17715532, 0.31811429, 0.32527842, 0.33320567, 0.37636394,
       0.52443438, 0.47098917, 0.47098917, 0.45964392, 0.48434

In [439]:
h1 = []
for i in range(int(len(pred_)/24)):      # 분 -> 시간 단위 바꾸기 위해
    h1.append(np.sum(pred_[24*i:24*(i+1)]))
h1

[7.169338313469801,
 6.8998659937126,
 7.728691873648879,
 12.024483248506785,
 8.146747587225637,
 7.065993870623592,
 6.414081999000436,
 6.430718104095468,
 5.379246387400356,
 6.106791437783615,
 8.23512751389336,
 8.09993745491429,
 6.454376535377891]

In [446]:
[a*30 for a in h1]

[215.08014940409402,
 206.99597981137802,
 231.86075620946636,
 360.73449745520355,
 244.4024276167691,
 211.97981611870776,
 192.42245997001308,
 192.92154312286405,
 161.3773916220107,
 183.20374313350845,
 247.0538254168008,
 242.9981236474287,
 193.63129606133674]

In [450]:
submission1['heat_supply_day1'] = [a*50 for a in h1]
submission1 

,yy,mm,dd,heat_supply_day1,heat_supply_day2
0,2022,3,4,358.466916,175.203884
1,2022,3,5,344.993300,180.095411
2,2022,3,6,386.434594,222.525890
3,2022,3,7,601.224162,375.730587
4,2022,3,8,407.337379,220.770130
5,2022,3,9,353.299694,177.159940
6,2022,3,10,320.704100,175.122535
7,2022,3,11,321.535905,202.310360
8,2022,3,12,268.962319,187.328093
9,2022,3,13,305.339572,128.252533


In [417]:
train_real

,10CM 정시 토양수분(%),20CM 정시 토양수분(%),30CM 정시 토양수분(%),50CM 정시 토양수분(%),일자,smart_farm.shield_light_h,smart_farm.shield_tmperature_h,smart_farm.shield_energy_h,smart_farm.shield_energy_v,smart_farm.exhaust_fan,smart_farm.ceiling,smart_farm.floating_fan,smart_farm.fan_coil_b_site,smart_farm.ventilation_temperature_control,smart_farm.heating_temperature_set_up,smart_farm.heat_supply
0,12.981818,16.572727,14.881818,18.472727,2022-03-04 00:00:00,0.0,0.000000,0.0,0.0,0.0,0.0,1.000000,0.866667,28.000000,18.000000,NaN
1,12.890909,16.536364,14.881818,18.481818,2022-03-04 01:00:00,0.0,0.000000,0.0,0.0,0.0,0.0,1.000000,0.733333,28.000000,18.000000,NaN
2,12.854545,16.509091,14.872727,18.481818,2022-03-04 02:00:00,0.0,0.000000,0.0,0.0,0.0,0.0,1.000000,0.866667,28.000000,18.000000,NaN
3,12.745455,16.490909,14.890909,18.490909,2022-03-04 03:00:00,0.0,0.000000,0.0,0.0,0.0,0.0,1.000000,0.816667,28.000000,18.000000,NaN
4,12.645455,16.472727,14.890909,18.490909,2022-03-04 04:00:00,0.0,0.000000,0.0,0.0,0.0,0.0,1.000000,0.866667,28.000000,18.000000,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
307,20.218182,21.172727,19.836364,22.627273,2022-03-20 19:00:00,0.0,91.316667,0.0,0.0,0.0,0.0,1.000000,0.000000,27.866667,17.383333,NaN
308,20.172727,21.109091,19.800000,22.627273,2022-03-20 20:00:00,0.0,0.200000,0.0,0.0,0.0,0.0,0.983333,0.000000,28.000000,15.478333,NaN
309,20.190909,21.072727,19.809091,22.609091,2022-03-20 21:00:00,0.0,0.000000,0.0,0.0,0.0,0.0,0.966667,0.150000,28.000000,15.095000,NaN
310,20.136364,21.045455,19.854545,22.618182,2022-03-20 22:00:00,0.0,0.000000,0.0,0.0,0.0,0.0,1.000000,0.616667,28.000000,16.616667,NaN


In [418]:
test_real

,10CM 정시 토양수분(%),20CM 정시 토양수분(%),30CM 정시 토양수분(%),50CM 정시 토양수분(%),일자,smart_farm.shield_light_h,smart_farm.shield_tmperature_h,smart_farm.shield_energy_h,smart_farm.shield_energy_v,smart_farm.exhaust_fan,smart_farm.ceiling,smart_farm.floating_fan,smart_farm.fan_coil_b_site,smart_farm.ventilation_temperature_control,smart_farm.heating_temperature_set_up,smart_farm.heat_supply
0,12.981818,16.572727,14.881818,18.472727,2022-03-04 00:00:00,0.0,0.000000,0.0,0.0,0.0,0.0,1.000000,0.866667,28.000000,18.000000,NaN
1,12.890909,16.536364,14.881818,18.481818,2022-03-04 01:00:00,0.0,0.000000,0.0,0.0,0.0,0.0,1.000000,0.733333,28.000000,18.000000,NaN
2,12.854545,16.509091,14.872727,18.481818,2022-03-04 02:00:00,0.0,0.000000,0.0,0.0,0.0,0.0,1.000000,0.866667,28.000000,18.000000,NaN
3,12.745455,16.490909,14.890909,18.490909,2022-03-04 03:00:00,0.0,0.000000,0.0,0.0,0.0,0.0,1.000000,0.816667,28.000000,18.000000,NaN
4,12.645455,16.472727,14.890909,18.490909,2022-03-04 04:00:00,0.0,0.000000,0.0,0.0,0.0,0.0,1.000000,0.866667,28.000000,18.000000,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
307,20.218182,21.172727,19.836364,22.627273,2022-03-20 19:00:00,0.0,91.316667,0.0,0.0,0.0,0.0,1.000000,0.000000,27.866667,17.383333,NaN
308,20.172727,21.109091,19.800000,22.627273,2022-03-20 20:00:00,0.0,0.200000,0.0,0.0,0.0,0.0,0.983333,0.000000,28.000000,15.478333,NaN
309,20.190909,21.072727,19.809091,22.609091,2022-03-20 21:00:00,0.0,0.000000,0.0,0.0,0.0,0.0,0.966667,0.150000,28.000000,15.095000,NaN
310,20.136364,21.045455,19.854545,22.618182,2022-03-20 22:00:00,0.0,0.000000,0.0,0.0,0.0,0.0,1.000000,0.616667,28.000000,16.616667,NaN


# Model2

In [423]:
X = train_real.drop(['일자','smart_farm.heat_supply'], axis=1)
y = train_real[['smart_farm.heat_supply']].values

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.3)

X_test = test_real.drop(['일자','smart_farm.heat_supply'], axis=1)

In [424]:
# X_train = train.drop(['smart_farm.yy', 'smart_farm.mm', 'smart_farm.dd', 'smart_farm.hh', 'smart_farm.nn', 'smart_farm.heat_supply'], axis=1)
# X_val = val.drop(['smart_farm.yy', 'smart_farm.mm', 'smart_farm.dd', 'smart_farm.hh', 'smart_farm.nn', 'smart_farm.heat_supply'],axis=1)

# y_train = train[['smart_farm.heat_supply']].values
# y_val = val[['smart_farm.heat_supply']].values

lgbm_reg = LGBMRegressor(objective= "regression",
                     max_depth= max_depth,
                     n_estimators= n_estimators,
                     learning_rate= lr,
                     num_leaves = num_leaves)
    
lgbm_reg.fit(X_train, y_train,
          eval_set=[(X_val, y_val)],
          eval_metric=["mse"],
          early_stopping_rounds=early_stopping_rounds,
          verbose=verbose) 


C:\Users\wjd72\anaconda3\envs\tensorflow_env\lib\site-packages\lightgbm\sklearn.py:726: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
C:\Users\wjd72\anaconda3\envs\tensorflow_env\lib\site-packages\lightgbm\sklearn.py:736: UserWarning: 'verbose' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose' argument is deprecated and will be removed in a future release of LightGBM. "
C:\Users\wjd72\anaconda3\envs\tensorflow_env\lib\site-packages\lightgbm\basic.py:179: UserWarning: Converting column-vector to 1d array
  _log_warning('Converting column-vector to 1d array')


[1]	valid_0's l2: 0.0477291
[2]	valid_0's l2: 0.0471948
[3]	valid_0's l2: 0.0466745
[4]	valid_0's l2: 0.0461618
[5]	valid_0's l2: 0.0456601
[6]	valid_0's l2: 0.0451685
[7]	valid_0's l2: 0.0446872
[8]	valid_0's l2: 0.0442155
[9]	valid_0's l2: 0.0437593
[10]	valid_0's l2: 0.0433062
[11]	valid_0's l2: 0.042863
[12]	valid_0's l2: 0.0424335
[13]	valid_0's l2: 0.0420078
[14]	valid_0's l2: 0.0415899
[15]	valid_0's l2: 0.0411867
[16]	valid_0's l2: 0.0407869
[17]	valid_0's l2: 0.0404009
[18]	valid_0's l2: 0.040021
[19]	valid_0's l2: 0.0396522
[20]	valid_0's l2: 0.0392969
[21]	valid_0's l2: 0.0389451
[22]	valid_0's l2: 0.0385963
[23]	valid_0's l2: 0.0382653
[24]	valid_0's l2: 0.0379309
[25]	valid_0's l2: 0.0375993
[26]	valid_0's l2: 0.0372854
[27]	valid_0's l2: 0.0369649
[28]	valid_0's l2: 0.0366696
[29]	valid_0's l2: 0.0363636
[30]	valid_0's l2: 0.0360605
[31]	valid_0's l2: 0.0357707
[32]	valid_0's l2: 0.0354841
[33]	valid_0's l2: 0.0352005
[34]	valid_0's l2: 0.034905
[35]	valid_0's l2: 0.03462

[641]	valid_0's l2: 0.0170659
[642]	valid_0's l2: 0.017065
[643]	valid_0's l2: 0.017062
[644]	valid_0's l2: 0.0170637
[645]	valid_0's l2: 0.0170627
[646]	valid_0's l2: 0.0170625
[647]	valid_0's l2: 0.0170628
[648]	valid_0's l2: 0.0170642
[649]	valid_0's l2: 0.0170642
[650]	valid_0's l2: 0.0170613
[651]	valid_0's l2: 0.017059
[652]	valid_0's l2: 0.0170564
[653]	valid_0's l2: 0.0170501
[654]	valid_0's l2: 0.0170498
[655]	valid_0's l2: 0.0170512
[656]	valid_0's l2: 0.0170521
[657]	valid_0's l2: 0.0170522
[658]	valid_0's l2: 0.0170494
[659]	valid_0's l2: 0.0170503
[660]	valid_0's l2: 0.0170481
[661]	valid_0's l2: 0.0170499
[662]	valid_0's l2: 0.0170499
[663]	valid_0's l2: 0.0170472
[664]	valid_0's l2: 0.0170448
[665]	valid_0's l2: 0.0170457
[666]	valid_0's l2: 0.0170477
[667]	valid_0's l2: 0.0170478
[668]	valid_0's l2: 0.0170453
[669]	valid_0's l2: 0.0170433
[670]	valid_0's l2: 0.0170424
[671]	valid_0's l2: 0.0170423
[672]	valid_0's l2: 0.01704
[673]	valid_0's l2: 0.0170391
[674]	valid_0's

[1463]	valid_0's l2: 0.0161163
[1464]	valid_0's l2: 0.0161157
[1465]	valid_0's l2: 0.0161118
[1466]	valid_0's l2: 0.0161123
[1467]	valid_0's l2: 0.0161118
[1468]	valid_0's l2: 0.016115
[1469]	valid_0's l2: 0.0161161
[1470]	valid_0's l2: 0.0161134
[1471]	valid_0's l2: 0.0161149
[1472]	valid_0's l2: 0.0161159
[1473]	valid_0's l2: 0.0161177
[1474]	valid_0's l2: 0.0161175
[1475]	valid_0's l2: 0.0161168
[1476]	valid_0's l2: 0.0161158
[1477]	valid_0's l2: 0.0161151
[1478]	valid_0's l2: 0.016115
[1479]	valid_0's l2: 0.0161155
[1480]	valid_0's l2: 0.0161166
[1481]	valid_0's l2: 0.0161134
[1482]	valid_0's l2: 0.0161142
[1483]	valid_0's l2: 0.0161124
[1484]	valid_0's l2: 0.0161124
[1485]	valid_0's l2: 0.0161138
[1486]	valid_0's l2: 0.0161133
[1487]	valid_0's l2: 0.0161156
[1488]	valid_0's l2: 0.0161168
[1489]	valid_0's l2: 0.0161154
[1490]	valid_0's l2: 0.0161109
[1491]	valid_0's l2: 0.0161103
[1492]	valid_0's l2: 0.0161085
[1493]	valid_0's l2: 0.0161102
[1494]	valid_0's l2: 0.0161066
[1495]	val

[2353]	valid_0's l2: 0.0158516
[2354]	valid_0's l2: 0.0158511
[2355]	valid_0's l2: 0.0158514
[2356]	valid_0's l2: 0.0158513
[2357]	valid_0's l2: 0.015855
[2358]	valid_0's l2: 0.0158534
[2359]	valid_0's l2: 0.0158493
[2360]	valid_0's l2: 0.0158476
[2361]	valid_0's l2: 0.0158465
[2362]	valid_0's l2: 0.0158461
[2363]	valid_0's l2: 0.0158499
[2364]	valid_0's l2: 0.0158492
[2365]	valid_0's l2: 0.0158495
[2366]	valid_0's l2: 0.0158482
[2367]	valid_0's l2: 0.015849
[2368]	valid_0's l2: 0.015848
[2369]	valid_0's l2: 0.0158465
[2370]	valid_0's l2: 0.0158517
[2371]	valid_0's l2: 0.0158511
[2372]	valid_0's l2: 0.0158471
[2373]	valid_0's l2: 0.0158461
[2374]	valid_0's l2: 0.015845
[2375]	valid_0's l2: 0.0158487
[2376]	valid_0's l2: 0.0158482
[2377]	valid_0's l2: 0.0158489
[2378]	valid_0's l2: 0.015849
[2379]	valid_0's l2: 0.0158485
[2380]	valid_0's l2: 0.0158538
[2381]	valid_0's l2: 0.0158523
[2382]	valid_0's l2: 0.0158507
[2383]	valid_0's l2: 0.0158516
[2384]	valid_0's l2: 0.0158494
[2385]	valid_

[3082]	valid_0's l2: 0.0157382
[3083]	valid_0's l2: 0.0157353
[3084]	valid_0's l2: 0.0157344
[3085]	valid_0's l2: 0.0157346
[3086]	valid_0's l2: 0.0157347
[3087]	valid_0's l2: 0.0157361
[3088]	valid_0's l2: 0.0157394
[3089]	valid_0's l2: 0.0157402
[3090]	valid_0's l2: 0.0157409
[3091]	valid_0's l2: 0.0157403
[3092]	valid_0's l2: 0.0157407
[3093]	valid_0's l2: 0.0157407
[3094]	valid_0's l2: 0.0157403
[3095]	valid_0's l2: 0.0157403
[3096]	valid_0's l2: 0.0157383
[3097]	valid_0's l2: 0.015739
[3098]	valid_0's l2: 0.0157374
[3099]	valid_0's l2: 0.0157376
[3100]	valid_0's l2: 0.0157377
[3101]	valid_0's l2: 0.015735
[3102]	valid_0's l2: 0.0157345
[3103]	valid_0's l2: 0.0157323
[3104]	valid_0's l2: 0.015733
[3105]	valid_0's l2: 0.0157322
[3106]	valid_0's l2: 0.0157315
[3107]	valid_0's l2: 0.0157328
[3108]	valid_0's l2: 0.0157324
[3109]	valid_0's l2: 0.0157305
[3110]	valid_0's l2: 0.0157306
[3111]	valid_0's l2: 0.0157306
[3112]	valid_0's l2: 0.0157295
[3113]	valid_0's l2: 0.0157265
[3114]	vali

[3963]	valid_0's l2: 0.015682
[3964]	valid_0's l2: 0.0156809
[3965]	valid_0's l2: 0.0156798
[3966]	valid_0's l2: 0.01568
[3967]	valid_0's l2: 0.0156795
[3968]	valid_0's l2: 0.0156795
[3969]	valid_0's l2: 0.0156786
[3970]	valid_0's l2: 0.0156794
[3971]	valid_0's l2: 0.0156796
[3972]	valid_0's l2: 0.0156788
[3973]	valid_0's l2: 0.0156791
[3974]	valid_0's l2: 0.0156786
[3975]	valid_0's l2: 0.0156782
[3976]	valid_0's l2: 0.0156768
[3977]	valid_0's l2: 0.0156783
[3978]	valid_0's l2: 0.0156821
[3979]	valid_0's l2: 0.015682
[3980]	valid_0's l2: 0.0156809
[3981]	valid_0's l2: 0.0156807
[3982]	valid_0's l2: 0.0156811
[3983]	valid_0's l2: 0.0156803
[3984]	valid_0's l2: 0.0156801
[3985]	valid_0's l2: 0.0156792
[3986]	valid_0's l2: 0.0156783
[3987]	valid_0's l2: 0.0156791
[3988]	valid_0's l2: 0.0156783
[3989]	valid_0's l2: 0.0156793
[3990]	valid_0's l2: 0.0156803
[3991]	valid_0's l2: 0.0156812
[3992]	valid_0's l2: 0.015681
[3993]	valid_0's l2: 0.015682
[3994]	valid_0's l2: 0.0156814
[3995]	valid_0

LGBMRegressor(learning_rate=0.01, max_depth=5, n_estimators=5000,
              num_leaves=100, objective='regression')

In [425]:
pred = lgbm_reg.predict(X_test)
pred                        

array([ 4.49891698e-01,  4.36044621e-01,  4.50479231e-01,  4.45980632e-01,
        4.46030078e-01,  5.02837263e-01,  5.01074530e-01,  4.84034330e-01,
        3.84010640e-01,  3.57451687e-02,  1.28705333e-01,  8.05284340e-02,
        4.14070967e-02,  4.56005365e-02,  1.78743309e-02,  2.67108727e-02,
        1.13491862e-01,  9.59651794e-02,  8.12150137e-02,  8.51560635e-02,
        1.03442172e-01,  2.19371104e-01,  3.47543608e-01,  3.16989676e-01,
        3.47195733e-01,  3.47534749e-01,  3.46375426e-01,  4.06340590e-01,
        4.06340590e-01,  5.27953275e-01,  5.23338730e-01,  4.96117356e-01,
        2.08590622e-01, -4.85309167e-02,  2.73341261e-02, -1.67261879e-02,
        1.27885640e-02,  1.36729460e-02,  2.91002415e-02, -9.06601204e-03,
       -1.75153825e-02,  1.52873536e-01,  2.85341072e-01,  3.08092665e-01,
        2.24434436e-01,  3.62839459e-01,  5.63484200e-01,  5.05270561e-01,
        5.21125081e-01,  4.80748066e-01,  5.21212962e-01,  5.05749269e-01,
        5.02157284e-01,  

In [432]:
h = []
for i in range(int(len(pred)/24)):      # 분 -> 시간 단위 바꾸기 위해
    h.append(np.sum(pred[24*i:24*(i+1)]))
h

[5.840129474629828,
 6.003180378948429,
 7.41752966938039,
 12.524352914621744,
 7.359004344110949,
 5.905331321733186,
 5.8374178395363145,
 6.743678677537618,
 6.244269779010166,
 4.2750844167271485,
 8.164982099205705,
 12.186185193968603,
 7.62239358917493]

In [451]:
# # pred = lgbm_reg.predict(X_test)
# submission=pd.read_csv('1-1_검증데이터셋.csv', encoding='utf-8')
# submission1=submission.copy()
submission1['heat_supply_day2'] = [a*50 for a in h]
submission1 
# submission1.to_csv("farm_lgbm.csv", index = False)            # 6위

,yy,mm,dd,heat_supply_day1,heat_supply_day2
0,2022,3,4,358.466916,292.006474
1,2022,3,5,344.993300,300.159019
2,2022,3,6,386.434594,370.876483
3,2022,3,7,601.224162,626.217646
4,2022,3,8,407.337379,367.950217
5,2022,3,9,353.299694,295.266566
6,2022,3,10,320.704100,291.870892
7,2022,3,11,321.535905,337.183934
8,2022,3,12,268.962319,312.213489
9,2022,3,13,305.339572,213.754221


In [452]:
submission1.to_csv('220224_soil.csv', index=False)

In [ ]:
## 다른 컬럼도 sum을 해서 줄지. 마지막에 50곱한건 논리설명 부족해서 다른 방안을 찾아야 함.!!